In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
anime_df = pd.read_csv("anime.csv")

# Fill missing values
anime_df['genre'].fillna('Unknown', inplace=True)
anime_df['type'].fillna('Unknown', inplace=True)
anime_df['episodes'] = pd.to_numeric(anime_df['episodes'], errors='coerce').fillna(0).astype(int)
anime_df['rating'].fillna(anime_df['rating'].mean(), inplace=True)


<ipython-input-4-340f02012a6f>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  anime_df['genre'].fillna('Unknown', inplace=True)
<ipython-input-4-340f02012a6f>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [ ]:
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(anime_df['genre'].str.split(', '))


In [ ]:
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(anime_df[['rating', 'members']])


In [ ]:
feature_matrix = pd.concat(
    [pd.DataFrame(genre_encoded, columns=mlb.classes_),
     pd.DataFrame(normalized_features, columns=['rating', 'members'])],
    axis=1
)


In [ ]:
cosine_sim = cosine_similarity(feature_matrix)


In [ ]:
def get_recommendations(anime_title, cosine_sim=cosine_sim):
    # Find the index of the anime with the given title
    idx = anime_df[anime_df['name'] == anime_title].index[0]

    # Get similarity scores for this anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the anime based on similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top 10 most similar anime (excluding itself)
    sim_scores = sim_scores[1:11]
    anime_indices = [i[0] for i in sim_scores]

    # Return the top 10 similar anime
    return anime_df.iloc[anime_indices][['name', 'rating', 'members']]


In [ ]:
recommendations = get_recommendations('Kimi no Na wa.')
print(recommendations)


                                                   name  rating  members
5805                        Wind: A Breath of Heart OVA    6.35     2043
6394                       Wind: A Breath of Heart (TV)    6.14     7778
1111              Aura: Maryuuin Kouga Saigo no Tatakai    7.67    22599
504   Clannad: After Story - Mou Hitotsu no Sekai, K...    8.02   138364
208                       Kokoro ga Sakebitagatterunda.    8.32    59652
1201                     Angel Beats!: Another Epilogue    7.63   134180
1435                                         True Tears    7.55   118644
1907                                   Myself; Yourself    7.41   115075
1631                                Kimikiss Pure Rouge    7.48    58211
2300                         Koi to Senkyo to Chocolate    7.30    91552


Interview Questions

1.User-Based Collaborative Filtering:

Recommends items based on the preferences of similar users,Identifies users with similar taste profiles to the target user (based on past ratings or interactions).

Item-Based Collaborative Filtering:

Recommends items based on their similarity to other items the user has liked.Determines how similar items are to each other based on user ratings or interactions.

2.Collaborative filtering is a powerful technique used in recommender systems to predict a user's preferences for items like movies, products, music based on the tastes of similar users or the characteristics of similar items. Collaborative filtering is a powerful tool for recommender systems, helping to enhance user experience and engagement by providing personalized suggestions. While it faces some challenges, it remains a cornerstone of modern recommendation algorithms